In [1]:
import os
import pandas as pd
import numpy as np
from functools import partial
import dproc, sgutil, sgnn, sgml

import seaborn as sns
import matplotlib.pyplot as plt

import shap

2024-12-24 06:15:51.439066: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1735020951.571328   46763 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1735020951.609713   46763 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-24 06:15:51.956433: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
data_path = 'data'
files = {
    k: os.path.join(p, f)
    for k, p, f in [
        ('train', data_path, 'train.csv'),
        ('test', data_path, 'test.csv'),
        ('org_train', data_path, 'train_org.csv'),
        ('org_test', data_path, 'test_org.csv'),
        ('train_parquet', data_path, 'train.parquet'),
        ('org_pkl', data_path, 'org.pkl'),
        ('test_parquet', data_path, 'test.parquet'),
        ('var_pkl', data_path, 'var.pkl')
    ]
}
df_train = pd.read_parquet(files['train_parquet']).set_index('id')
sc = sgutil.SGCache('img', 'result')
pd_procs = dproc.PD_Vars.load(os.path.join('data', 'vars') )
target = 'Response'
df_train, _ = pd_procs.procs_all(df_train)
#del df
pd_procs.df_var

,Description,src,na,count,n_unique,dtype
Age,None,None,0.0,0.0,0.0,Int8
Age_S,Age의 범주형 변수,cat_proc,0.0,11504798.0,66.0,Categorical
Annual_Premium,None,None,0.0,0.0,0.0,Float32
Annual_Premium_S,Annual_Premium의 범주형 변수,cat_proc,0.0,11504798.0,43119.0,Categorical
Driving_License,None,None,0.0,0.0,0.0,Int8
Gender,None,None,0.0,0.0,0.0,Categorical(ordering='physical')
Policy_Sales_Channel,None,None,0.0,0.0,0.0,Categorical(ordering='physical')
Policy_Sales_Channel_S,Policy_Sales_Channel의 범주형 변수,cat_proc,0.0,11504798.0,141.0,Categorical
Previously_Insured,None,None,0.0,0.0,0.0,Categorical(ordering='physical')
Region_Code,None,None,0.0,0.0,0.0,Categorical(ordering='physical')


In [3]:
from sklearn.preprocessing import TargetEncoder
from sklearn.model_selection import StratifiedKFold

def tgt_proc(df):
    tgt_list = ['VP', 'VPV', 'VPVi', 'VPA', 'VPAg', 'VPP', 'VPR', 'VPVP', 'VPPAg', 'VPPR', 'VPVA', 'VPAgA']
    tgt_agg = {
        i : list()
        for i in tgt_list
    }
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)
    for train_idx, valid_idx in skf.split(df, df[target]):
        df_cv_train, df_valid = df.iloc[train_idx], df.iloc[valid_idx]
        for i in tgt_list:
            tgt_agg[i].append(
                pd.Series(
                    TargetEncoder(random_state=123).fit(df_cv_train[[i]], df_cv_train[target]).transform(df_valid[[i]])[:, 0],
                    index = df_valid[i].index
                )
            )
    return {
        i: pd.concat(tgt_agg[i])
        for i in tgt_list
    }
tgt = sc.cache_result(
    'tgt', lambda : tgt_proc(df_train)
)
df_train['D1'] = tgt['VPVi'] - tgt['VP']
df_train['D2'] = tgt['VPA'] - tgt['VP']
df_train['D3'] = tgt['VPAg'] - tgt['VP']
df_train['D4'] = tgt['VPP'] - tgt['VP']
df_train['D5'] = tgt['VPR'] - tgt['VP']
df_train['D6'] = tgt['VPVP'] - tgt['VPV']
df_train['D7'] = tgt['VPPAg'] - tgt['VPP']
df_train['D8'] = tgt['VPPR'] - tgt['VPP']
df_train['D9'] = tgt['VPVA'] - tgt['VPV']
df_train['D10'] = tgt['VPAgA'] - tgt['VPAg']

del tgt

In [4]:
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

import lightgbm as lgb
import xgboost as xgb
import catboost as cb
import sgml

def get_validation_splitter(validation_fraction):
    return lambda x: train_test_split(x, test_size=validation_fraction, stratify=x[target])

config = {
    'predict_func': lambda m, df, X: pd.Series(m.predict_proba(df[X])[:, 1], index=df.index),
    'score_func': lambda df, prds: roc_auc_score(df[target].sort_index(), prds.sort_index()),
    'validation_splitter': get_validation_splitter,
    'progress_callback': sgml.ProgressCallBack(),
    'return_train_scores': False,
    'y': target,
}

lgb_adapter = sgml.LGBMAdapter(model=lgb.LGBMClassifier)
xgb_adapter = sgml.XGBAdapter(model=xgb.XGBClassifier) 
cb_adapter = sgml.CBAdapter(model=cb.CatBoostClassifier)
lr_adapter = sgml.SklearnAdapter(LogisticRegression)


def print_metrics(title, m):
    print("roc_auc, {}: {:.5f}±{:.5f}".format(
        title, np.mean(m), np.std(m)
    ))

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)
ss = StratifiedShuffleSplit(n_splits=1, train_size=0.8, random_state=123)

In [5]:
import tensorflow as tf
import sgnn
from functools import partial

class ResponseModel(tf.keras.Model):
    def __init__(self, emb_cols, cont_cols, f_size, left_layers, right_layers, top_layers , **argv):
        super().__init__()
        if emb_cols != None:
            self.emb_layers = {
                v: tf.keras.layers.Embedding(c, s, name=v) for v, c, s in emb_cols
            }
            self.emb_cc = tf.keras.layers.Concatenate(axis=-1)
            if left_layers != None:
                self.left_model = tf.keras.models.Sequential([
                    tf.keras.layers.Dense(**params) for params in left_layers
                ])
                self.right_model = tf.keras.models.Sequential([
                    tf.keras.layers.Dense(**params) for params in right_layers
                ])
                self.mul = tf.keras.layers.Multiply()
                self.left_slice_layer_1 = tf.keras.layers.Lambda(lambda x: x[:, :f_size])
                self.left_slice_layer_2 = tf.keras.layers.Lambda(lambda x: x[:, f_size:])
                self.right_slice_layer_1 = tf.keras.layers.Lambda(lambda x: x[:, :f_size])
                self.right_slice_layer_2 = tf.keras.layers.Lambda(lambda x: x[:, f_size:])
            else:
                self.left_model = None
        else:
            self.emb_layers = None
        self.top_cc = tf.keras.layers.Concatenate(axis=-1)
        self.top_model = tf.keras.models.Sequential([
            tf.keras.layers.Dense(**params) for params in top_layers
        ])
        self.f_size = f_size
        self.cont_cols = cont_cols
        self.sigmoid = tf.keras.layers.Activation('sigmoid')

    def __call__(self, X):
        if self.emb_layers != None:
            X_emb = self.emb_cc([tf.squeeze(v(X[k]), axis=-2) for k, v in self.emb_layers.items()])
            if self.left_model !=None:
                X_left = self.left_model(X_emb)
                X_right = self.right_model(X_emb)
                X_left_1 = self.left_slice_layer_1(X_left)
                X_left_2 = self.left_slice_layer_2(X_left)
                
                X_right_1 = self.right_slice_layer_1(X_right)
                X_right_2 = self.right_slice_layer_2(X_right)
                X_mul = self.mul([X_left_1, X_right_1])
                cc_list = [X_left_2, X_right_2, X_mul]
            else:
                cc_list = [X_emb]
        else:
            cc_list = list()
        if self.cont_cols != None:
            cc_list.append(X[self.cont_cols])
        if len(cc_list) == 1:
            return self.sigmoid(
                self.top_model(cc_list[0])
            )
        else:
            return self.sigmoid(
                self.top_model(self.top_cc(cc_list))
            )

def to_tf_dataset(X, Y=None, sample_weights=None, cont=[], cat=[]):
    d = {}
    for i, n in enumerate(cat):
        d[n] = np.expand_dims(X[:, i], axis=-1)
    if len(cont) > 0:
        d['Continous'] =  X[:, (i + 1):]
    
    if Y is None:
        return tf.data.Dataset.from_tensor_slices(d)
    else:
        if type(Y) == pd.Series:
            if sample_weights is None:
                return tf.data.Dataset.from_tensor_slices((d, Y.values))
            else:
                return tf.data.Dataset.from_tensor_slices((d, Y.values, sample_weights.values))
        else:
            if sample_weights is None:
                return tf.data.Dataset.from_tensor_slices((d, Y))
            else:
                return tf.data.Dataset.from_tensor_slices((d, Y, sample_weights))

def nn_cat_param(df, name, size):
    return name, len(df[name].cat.categories), size

nn_adapter = sgnn.NNAdapter(
    sgnn.NNClassifier, 
    to_tf_dataset=partial(to_tf_dataset, 
            cont=['mm__{}'.format(i) for i in ['Annual_Premium', 'Vintage', 'Age', 'Driving_License']], 
            cat= ['Annual_Premium_S', 'Vintage_S', 'Vehicle_Damage', 'Previously_Insured', 'Policy_Sales_Channel_S', 'Age_S', 'Vehicle_Age', 'Region_Code_S', 
                'Gender', 'VA_Age']
    )
)

# LR

In [6]:
lr = sgml.CVModel.load_or_create('result', 'LR', skf, config, lr_adapter)

In [7]:
hparams = {
    'model_params': {},
    'X_tgt': ['Annual_Premium_S'],
    'X_mm': ['Annual_Premium', 'D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9', 'D10'], 
    'X_ohe': ['Vehicle_Damage', 'Previously_Insured', 'Policy_Sales_Channel_S', 'Age_S', 'Vehicle_Age', 'Region_Code_S', 
              'Gender', 'Driving_License'],
    'ohe': {'drop': 'first', 'handle_unknown': 'ignore'},
}
result = lr.cv(df_train, hparams)
np.mean(result['valid_scores'])

0.8894339951376733

# CB1

In [8]:
cb1 = sgml.CVModel.load_or_create('result', 'CB1', skf, config, cb_adapter)

In [9]:
hparams = {
    'model_params': {'n_estimators': 1200, 'learning_rate': 0.15, 'max_depth': 9},
    'X_num': ['Annual_Premium', 'Age', 'D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9', 'D10'], 
    'X_cat': ['Annual_Premium_S', 'Vehicle_Damage', 'Previously_Insured', 'Policy_Sales_Channel_S', 'Age_S', 'Vehicle_Age', 'Region_Code_S', 
              'Gender', 'Driving_License'],
}
result = cb1.cv(df_train, hparams, task_type='GPU')
result['valid_scores']

[0.91197412898988,
 0.9128653591163912,
 0.9130351624263097,
 0.9129334930848559,
 0.9128286380114352]

# LGB1

In [10]:
lgb1 = sgml.CVModel.load_or_create('result', 'LGB1', skf, config, lgb_adapter)

In [11]:
c_type = pd.CategoricalDtype(['etc'] + df_train['VPA'].value_counts().pipe(lambda x: x.loc[x >= 10].index.tolist()))
df_train['VPA2'] = dproc.rearrange_cat(df_train['VPA'], c_type, lambda c, v: 0 if v not in c else v, use_set = True)

In [12]:
hparams = {
    'model_params': {'n_estimators': 1000, 'learning_rate': 0.01, 'num_leaves': 255},
    'X_tgt': ['Annual_Premium_S'],
    'X_num': ['Annual_Premium', 'Age', 'D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9', 'D10'], 
    'X_cat': ['Vehicle_Damage', 'Previously_Insured', 'Policy_Sales_Channel_S', 'Age_S', 'Vehicle_Age', 'Region_Code_S', 
              'Gender', 'Driving_License'],
    'validation_fraction': 0.1
}

#result = lgb1.adhoc(df_train, ss, hparams)
#result['valid_scores'], result['train_scores']

# XGB1

In [13]:
xgb1 = sgml.CVModel.load_or_create('result', 'XGB1', skf, config, xgb_adapter)

In [14]:
hparams = {
    'model_params': {'n_estimators': 1000, 'learning_rate': 0.05, 'max_depth': 8},
    'X_tgt': ['Annual_Premium_S'],
    'X_num': ['Annual_Premium', 'Age', 'D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9'], 
    'X_ohe': ['Vehicle_Damage', 'Previously_Insured', 'Policy_Sales_Channel_S', 'Age_S', 'Vehicle_Age', 'Region_Code_S', 
              'Gender', 'Driving_License'],
    'ohe': {'drop': 'if_binary'}
}

#result = lgb1.cv(df_train, hparams, xgb_adapter, device='cuda')
#result['valid_scores'], result['train_scores']

# NN1

In [15]:
# nn = sgml.CVModel.load_or_create('result', 'NN1', skf, config, nn_adapter)

In [16]:
"""X_nn_emb = [
    ('Annual_Premium_S', 4), ('Vehicle_Damage', 1), ('Previously_Insured', 1), ('Policy_Sales_Channel_S', 3), ('Age_S', 3), ('Vehicle_Age', 1), ('Region_Code_S', 2), 
    ('Gender', 1)
]
hparams = {
    'model_params': {
        'model': ResponseModel,
        'model_params': {
            'emb_cols': [
                nn_cat_param(df_train, v, n) for v, n in X_nn_emb
            ], 
            'cont_cols':  'Continous',
            'f_size':  32, 
            'left_layers': [
                {'units': 64, 'activation': 'relu', 'kernel_initializer': 'he_uniform'}, 
                {'units': 64, 'activation': 'relu', 'kernel_initializer': 'he_uniform'}, 
                {'units': 64, 'activation': 'relu', 'kernel_initializer': 'he_uniform'}, 
                {'units': 64, 'kernel_initializer': 'glorot_uniform'}]
            , 
            'right_layers': [
                {'units': 64, 'activation': 'relu', 'kernel_initializer': 'he_uniform'}, 
                {'units': 64, 'activation': 'relu', 'kernel_initializer': 'he_uniform'}, 
                {'units': 64, 'activation': 'relu', 'kernel_initializer': 'he_uniform'}, 
                {'units': 32, 'kernel_initializer': 'glorot_uniform'}
            ], 
            'top_layers': [
                {'units': 32, 'activation': 'relu', 'kernel_initializer': 'he_uniform'}, 
                {'units': 16, 'activation': 'relu', 'kernel_initializer': 'he_uniform'}, 
                {'units': 1, 'kernel_initializer': 'glorot_uniform'}
            ]
        },
        'batch_size': 1024,
        'shuffle_size': 204800,
        'optimizer': ('Adam', {'learning_rate': 0.0005}),
        'epochs': 5
    },
    'X_mm': ['Annual_Premium', 'Age', 'Driving_License', 'D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9'], 
    'X_cat': ['Annual_Premium_S', 'Vehicle_Damage', 'Previously_Insured', 'Policy_Sales_Channel_S', 'Age_S', 'Vehicle_Age', 'Region_Code_S', 
              'Gender'],
}"""
# result = nn.cv(df_train, hparams, progress_callback=sgml.ProgressCallBack())
# np.mean(result['valid_scores']), np.mean(result['train_scores'])
''

''

In [17]:
cb1.train(df_train, task_type='GPU')

{'variables': array(['cat__Annual_Premium_S', 'cat__Vehicle_Damage',
        'cat__Previously_Insured', 'cat__Policy_Sales_Channel_S',
        'cat__Age_S', 'cat__Vehicle_Age', 'cat__Region_Code_S',
        'cat__Gender', 'cat__Driving_License', 'pt__Annual_Premium',
        'pt__Age', 'pt__D1', 'pt__D2', 'pt__D3', 'pt__D4', 'pt__D5',
        'pt__D6', 'pt__D7', 'pt__D8', 'pt__D9', 'pt__D10'], dtype=object),
 'train_shape': (11504798, 21),
 'target': 'Response',
 'target_func': None}

In [18]:
df_test = pd.read_parquet(files['test_parquet']).set_index('id')
df_test, _ = pd_procs.procs_all(df_test)

In [ ]:
tgt_list = ['VP', 'VPV', 'VPVi', 'VPA', 'VPAg', 'VPP', 'VPR', 'VPVP', 'VPPAg', 'VPPR', 'VPVA', 'VPAgA']
df_tgt_test = sc.cache_result(
    'tgt_test',
    lambda : pd.concat([
        pd.Series(
            TargetEncoder(random_state=123).fit(df_train[[i]], df_train[target]).transform(df_test[[i]])[:, 0],
            index = df_test[i].index, name= i
        ) for i in tgt_list
    ], axis=1)
)

df_test['D1'] = df_tgt_test['VPVi'] - df_tgt_test['VP']
df_test['D2'] = df_tgt_test['VPA'] - df_tgt_test['VP']
df_test['D3'] = df_tgt_test['VPAg'] - df_tgt_test['VP']
df_test['D4'] = df_tgt_test['VPP'] - df_tgt_test['VP']
df_test['D5'] = df_tgt_test['VPR'] - df_tgt_test['VP']
df_test['D6'] = df_tgt_test['VPVP'] - df_tgt_test['VPV']
df_test['D7'] = df_tgt_test['VPPAg'] - df_tgt_test['VPP']
df_test['D8'] = df_tgt_test['VPPR'] - df_tgt_test['VPP']
df_test['D9'] = df_tgt_test['VPVA'] - df_tgt_test['VPV']
df_test['D10'] = df_tgt_test['VPAgA'] - df_tgt_test['VPAg']

del df_tgt_test

In [20]:
del df_tgt_test

In [21]:
cb1.get_predictor()(df_test)

id
11504798    0.005462
11504799    0.708622
11504800    0.439974
11504801    0.000117
11504802    0.079150
              ...   
19174659    0.160997
19174660    0.000316
19174661    0.000522
19174662    0.717448
19174663    0.000207
Length: 7669866, dtype: float64